In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [53]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train (1).csv
Saving predict.csv to predict (1).csv
Saving example.csv to example (2).csv


In [54]:
train = pd.read_csv('train.csv')
predict = pd.read_csv('predict.csv')
example = pd.read_csv('example.csv')

In [55]:
train.head(10)

,Vmag,Plx,e_Plx,B-V,SpType,Amag,TargetClass
0,6.901420,24.569658,0.862829,0.737416,G5V,20.449126,1
1,4.334196,2.198947,0.593309,1.153210,G2Ib,13.043324,0
2,7.873019,11.750465,0.616568,0.589166,F5V,17.736492,1
3,8.110014,2.143815,0.848508,1.322107,K2III,15.592469,0
4,8.025193,4.165622,0.899296,1.141646,K5III,16.379680,0
5,2.899130,11.110086,0.701719,1.095357,M0III,14.370388,0
6,7.529711,9.160176,1.131393,0.926519,G8IV,15.938899,1
7,6.083824,3.928857,0.835288,1.761544,M2.5IIIa,14.063334,0
8,3.000914,5.713252,0.757593,0.113611,A0V,14.350210,1
9,8.706528,1.816360,1.409039,0.250017,A0V,14.763104,1


In [56]:
len(train['SpType'].unique())

1842

In [57]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31641 entries, 0 to 31640
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Vmag         31641 non-null  float64
 1   Plx          31641 non-null  float64
 2   e_Plx        31641 non-null  float64
 3   B-V          31641 non-null  float64
 4   SpType       31641 non-null  object 
 5   Amag         31641 non-null  float64
 6   TargetClass  31641 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 1.7+ MB


In [60]:
import re

In [65]:
for index, row in train.iterrows():
  train.loc[index, 'first_class']= str(re.findall(r'.*\d', train.loc[index, 'SpType']))
  train.loc[index, 'second_class']=str(re.split(r'.*\d', train.loc[index, 'SpType']))
train

,Vmag,Plx,e_Plx,B-V,SpType,Amag,TargetClass,first_class,second_class
0,6.901420,24.569658,0.862829,0.737416,G5V,20.449126,1,['G5'],"['', 'V']"
1,4.334196,2.198947,0.593309,1.153210,G2Ib,13.043324,0,['G2'],"['', 'Ib']"
2,7.873019,11.750465,0.616568,0.589166,F5V,17.736492,1,['F5'],"['', 'V']"
3,8.110014,2.143815,0.848508,1.322107,K2III,15.592469,0,['K2'],"['', 'III']"
4,8.025193,4.165622,0.899296,1.141646,K5III,16.379680,0,['K5'],"['', 'III']"
...,...,...,...,...,...,...,...,...,...
31636,7.561113,0.692340,0.843319,0.252874,A0II,12.542120,0,['A0'],"['', 'II']"
31637,8.078011,3.002784,0.689891,0.971966,G8III,15.379710,0,['G8'],"['', 'III']"
31638,7.065964,7.820498,0.885794,0.435381,F3IV/V,16.737442,1,['F3'],"['', 'IV/V']"
31639,8.685452,2.643033,1.064647,0.988600,K0III,14.859882,0,['K0'],"['', 'III']"


In [66]:
le = LabelEncoder()
label = le.fit_transform(train['first_class'])
label

array([131, 125, 100, ...,  95, 149, 127])

In [69]:
train.drop("first_class", axis=1, inplace=True)
train["first_class"] = label

In [72]:
le = LabelEncoder()
label = le.fit_transform(train['second_class'])
label

array([302, 268, 302, ..., 195,  91, 302])

In [84]:
train.drop("second_class", axis=1, inplace=True)
train["second_class"] = label

In [103]:
train=train.drop(['SpType'],axis=1)

KeyError: ignored

In [87]:
train.head()

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass,first_class,second_class
0,6.901420,24.569658,0.862829,0.737416,20.449126,1,131,302
1,4.334196,2.198947,0.593309,1.153210,13.043324,0,125,268
2,7.873019,11.750465,0.616568,0.589166,17.736492,1,100,302
3,8.110014,2.143815,0.848508,1.322107,15.592469,0,160,91
4,8.025193,4.165622,0.899296,1.141646,16.379680,0,175,91


In [88]:
X = train.drop(['TargetClass'], axis = 1)
y = train['TargetClass']

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_st = scaler.fit_transform(X)

X_st

array([[-0.4333995 ,  2.92128821, -0.48118221, ...,  1.87080469,
         0.3484485 ,  1.13362612],
       [-1.90593956, -0.60937271, -1.25563994, ..., -1.28118598,
         0.23230573,  0.79982232],
       [ 0.12390262,  0.89809721, -1.18880461, ...,  0.71627778,
        -0.25162248,  1.13362612],
       ...,
       [-0.33901783,  0.27784946, -0.41519425, ...,  0.29107101,
        -0.34840812,  0.08312593],
       [ 0.58990789, -0.53928478,  0.09873588, ..., -0.50803902,
         0.6968768 , -0.93792099],
       [ 0.54805615,  2.19375992, -0.74322194, ...,  2.0262486 ,
         0.27101998,  1.13362612]])

In [89]:
train['TargetClass'].value_counts()

0    15842
1    15799
Name: TargetClass, dtype: int64

In [90]:
y = train['TargetClass']
X = train.drop(columns = ['TargetClass'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

In [91]:
%%time
rfc = RandomForestClassifier(random_state = 42, n_jobs = -1)
rfc.fit(X_train, y_train)

CPU times: user 4.53 s, sys: 14.1 ms, total: 4.55 s
Wall time: 2.45 s


RandomForestClassifier(n_jobs=-1, random_state=42)

In [92]:
pred = rfc.predict(X_test)

In [93]:
roc_auc_score(y_test, pred)

0.9914670299867816

In [94]:
predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7911 entries, 0 to 7910
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Vmag    7911 non-null   float64
 1   Plx     7911 non-null   float64
 2   e_Plx   7911 non-null   float64
 3   B-V     7911 non-null   float64
 4   SpType  7911 non-null   object 
 5   Amag    7911 non-null   float64
dtypes: float64(5), object(1)
memory usage: 371.0+ KB


In [96]:
for index, row in predict.iterrows():
  predict.loc[index, 'first_class']= str(re.findall(r'.*\d', predict.loc[index, 'SpType']))
  predict.loc[index, 'second_class']=str(re.split(r'.*\d', predict.loc[index, 'SpType']))
predict

,Vmag,Plx,e_Plx,B-V,SpType,Amag,first_class,second_class
0,4.039519,13.596504,0.845101,-0.097823,F5V+...,15.774122,['F5'],"['', 'V+...']"
1,6.655719,4.010332,1.050017,1.386670,K5III,15.344976,['K5'],"['', 'III']"
2,6.405060,0.348855,0.961598,0.951503,B0IVpe,12.236103,['B0'],"['', 'IVpe']"
3,9.113068,2.954461,1.251422,1.002819,K1III,15.560838,['K1'],"['', 'III']"
4,6.107686,3.144180,0.475795,0.493345,G8II-IIIvar,12.771981,['G8'],"['', 'II-IIIvar']"
...,...,...,...,...,...,...,...,...
7906,8.255272,2.858885,0.822289,1.349195,M3III,16.438014,['M3'],"['', 'III']"
7907,6.255358,3.034436,0.449386,0.966053,K0III,13.898403,['K0'],"['', 'III']"
7908,8.643598,10.932145,0.947897,0.504383,G0V,18.980729,['G0'],"['', 'V']"
7909,9.400810,6.279361,1.332124,0.351073,F2V,16.872491,['F2'],"['', 'V']"


In [98]:
label = le.fit_transform(predict['second_class'])
label

array([204,  62, 151, ..., 202, 202,  62])

In [97]:
label = le.fit_transform(predict['first_class'])
label

array([ 74, 130,  29, ...,  88,  66, 117])

In [100]:
predict.drop("first_class", axis=1, inplace=True)
predict["first_class"] = label

In [101]:
predict.drop("second_class", axis=1, inplace=True)
predict["second_class"] = label

In [104]:
predict=predict.drop(['SpType'],axis=1)

In [107]:
X = predict
y = train['TargetClass']

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_st = scaler.fit_transform(X)

X_st

array([[-2.04553489,  1.15327553, -0.54425455, ..., -0.13089798,
         1.14598635,  1.14598635],
       [-0.56977334, -0.33321519,  0.0549465 , ..., -0.31256157,
        -0.93823274, -0.93823274],
       [-0.71116615, -0.90098634, -0.2036028 , ..., -1.62859304,
         0.36807359,  0.36807359],
       ...,
       [ 0.55156168,  0.74012357, -0.24366539, ...,  1.22650585,
         1.11663116,  1.11663116],
       [ 0.97869405,  0.01863439,  0.87986424, ...,  0.33405769,
         1.11663116,  1.11663116],
       [ 0.63558406, -0.58613478,  1.22838188, ..., -0.14639816,
        -0.93823274, -0.93823274]])

In [108]:
pred = rfc.predict(predict)

In [109]:
pred = pd.Series(pred)

In [110]:
pred.head(10)

0    1
1    0
2    0
3    0
4    1
5    1
6    1
7    0
8    1
9    1
dtype: int64

In [111]:
pred.to_csv('Result.csv')
pred_file = pd.read_csv('Result.csv')
pred_file.head(10)

,Unnamed: 0,0
0,0,1
1,1,0
2,2,0
3,3,0
4,4,1
5,5,1
6,6,1
7,7,0
8,8,1
9,9,1


In [112]:
pred_file.rename(columns={'0': 'TargetClass','Unnamed: 0':'Number'}, inplace=True)

In [113]:
pred_file.head(10)

,Number,TargetClass
0,0,1
1,1,0
2,2,0
3,3,0
4,4,1
5,5,1
6,6,1
7,7,0
8,8,1
9,9,1


In [114]:
predict=predict.join(pred_file)

In [115]:
predict.head()

,Vmag,Plx,e_Plx,B-V,Amag,first_class,second_class,Number,TargetClass
0,4.039519,13.596504,0.845101,-0.097823,15.774122,204,204,0,1
1,6.655719,4.010332,1.050017,1.386670,15.344976,62,62,1,0
2,6.405060,0.348855,0.961598,0.951503,12.236103,151,151,2,0
3,9.113068,2.954461,1.251422,1.002819,15.560838,62,62,3,0
4,6.107686,3.144180,0.475795,0.493345,12.771981,36,36,4,1


In [116]:
pred_final = predict.drop(['Number'], axis='columns')

In [117]:
pred_final.head(10)

,Vmag,Plx,e_Plx,B-V,Amag,first_class,second_class,TargetClass
0,4.039519,13.596504,0.845101,-0.097823,15.774122,204,204,1
1,6.655719,4.010332,1.050017,1.386670,15.344976,62,62,0
2,6.405060,0.348855,0.961598,0.951503,12.236103,151,151,0
3,9.113068,2.954461,1.251422,1.002819,15.560838,62,62,0
4,6.107686,3.144180,0.475795,0.493345,12.771981,36,36,1
5,7.337771,12.254347,0.651041,0.330493,17.315567,202,202,1
6,7.316560,0.662464,0.720464,-0.056743,12.845317,212,212,1
7,7.379830,4.494505,0.594863,1.322639,15.528721,62,62,0
8,9.284617,7.392761,1.116969,0.629743,19.626463,202,202,1
9,8.646860,4.137044,1.239575,0.371263,17.284021,208,208,1


In [118]:
pred_final.to_csv('Result_final.csv',index=False)
files.download("Result_final.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>